In [ ]:
import pandas as pd
import json
import glob
import os
import geopandas as gpd

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)


In [ ]:
 
# WWF ECOREGION AND USA COUNTY MAP DATA

# read in ecoregion map data and US county data
counties = gpd.read_file(
    '/home/muskrat/Documents/eco_data_copy/wwf_map_data/us_counties.geojson')
ecomap = gpd.read_file(
    '/home/muskrat/Documents/eco_data_copy/wwf_map_data/map.geojson')

# ecotest = ecomap.loc[ecomap['unique_id'] == 420].reset_index(drop=True)
# county_inner = counties.loc[0]
# county_outer = counties.loc[1]
# county_split = counties.loc[(counties['NAME'] == 'Brown') & (counties['STATE_NAME'] == 'Ohio')]


eco_county = []
# eco_id = {'unique_id': str(ecotest['unique_id'].values[0])}
# eco_county.append(eco_id)
# selected_counties = []

# check whether a each county is inside or intersects an ecoregion or if ecoregion is inside a county
for i in range(len(ecomap)):
    # get unique id of each ecoregion
    eco_id = {'unique_id': str(ecomap.loc[i, 'unique_id'])}

    selected_counties = []

    for j in range(len(counties)):
        # check whether county is within ecoregion
        # contains = ecomap.loc[i, 'geometry'].contains(counties.loc[j, 'geometry'])
        # intersects = ecomap.loc[i, 'geometry'].intersects(counties.loc[j, 'geometry'])
        within = ecomap.loc[i, 'geometry'].within(counties.loc[j, 'geometry'])

        # print(counties.loc[i, 'NAME'])
        # if county is in ecoregion push to selected counties list
        # if contains == True:
        if within == True:
            # if contains == True or intersects == True:
            selected_counties.append({
                'county': counties.loc[j, 'NAME'],
                'state': counties.loc[j, 'STATE_NAME']})
            # eco_id['eco_counties'] = selected_counties
       # add selected counties to eco_id dict
    eco_id['eco_counties'] = selected_counties
    # push counties to eco_county list
    eco_county.append(eco_id)


full_county = []

# check if there are any counties in ecoregion for usa. if they are push to full county list
for i in range(len(eco_county)):
    if eco_county[i]['eco_counties']:
        full_county.append(eco_county[i])

na_county = []

# check if ecoregion has unique_id. if it does push to na_county
for i in range(len(full_county)):
    if full_county[i]['unique_id'] != 'nan':
        na_county.append(full_county[i])

county_data = pd.DataFrame(na_county)
eco_county_data = pd.json_normalize(na_county,
                                   record_path='eco_counties',
                                   meta=['unique_id'])

# Extract relevant columns
county_data = eco_county_data[['unique_id', 'county']]
state_data = eco_county_data[['unique_id', 'state']]

# Group by unique_id
county_group = county_data.groupby('unique_id')['county'].apply(list).reset_index()
state_group = state_data.groupby('unique_id')['state'].apply(list).reset_index()

# Merge grouped data
final_eco_county = pd.merge(county_group, state_group, on='unique_id', how='left')

# Convert to JSON
eco_county_json = final_eco_county.to_json(orient='records', force_ascii=False)



file = open("eco_county_prep.json", "w")
file.write(eco_county_json)
file.close



In [ ]:


# USA ECO COUNTIES

eco_county = pd.read_json('/home/muskrat/Desktop/eco_county_prep.json')

# convert to dataframe and group county and state into dict.
county = ['county', 'state']
eco_county['eco_counties'] = eco_county[county].to_dict(orient='records')

# group by unique id
eco_group = eco_county.groupby(
    'unique_id')['eco_counties'].apply(list).reset_index()
# eco_group = eco_county.groupby('unique_id')['eco_counties']

# convert to list
eco_counties = eco_group['eco_counties'].to_list()

eco_counties_test = eco_counties

# remove duplicates from list of counties and states
for j in range(len(eco_counties_test)):

    eco_counties_test[j] = [i for n, i in enumerate(
        eco_counties_test[j]) if i not in eco_counties_test[j][n + 1:]]


eco_counties_df = pd.Series(eco_counties_test, name='counties_final')

group_final = eco_group.merge(eco_counties_df, left_index=True, right_index=True).reindex(
    columns=['unique_id', 'counties_final'])

group_final_json = group_final.to_json(orient='records', force_ascii=False)
repr(group_final_json)

file = open("eco_county_within.json", "w")
file.write(group_final_json)
file.close


In [ ]:



 
# USA ECO COUNTIES

# eco_county_contains = pd.read_json(
#     '/home/muskrat/Documents/eco_data_copy/main_eco_data/eco_county_contains.json')
# eco_county_intersects = pd.read_json(
#     '/home/muskrat/Documents/eco_data_copy/main_eco_data/eco_county_intersects.json')
# eco_county_within = pd.read_json(
#     '/home/muskrat/Documents/eco_data_copy/main_eco_data/eco_county_within.json')


 
# USA ECO COUNTIES

eco_ods = pd.read_excel('/home/muskrat/Documents/eco_data_copy/US_county_plants/397/397.ods',
                        index_col=0, sheet_name=None, engine=('odf'))

# put all sheets in one dataframe
eco_df = pd.concat([v for k, v in eco_ods.items()])
eco_df.reset_index(inplace=True)

# add unique id column with ecoregion value
eco_df['unique_id'] = '397'

 
# USA ECO COUNTIES
# remove duplicate species
eco_df.drop_duplicates(['Scientific Name'], inplace=True, ignore_index=True)

 
# USA ECO COUNTIES

eco_subset = eco_df[['Plant Type', 'Scientific Name', 'Common Name',
                'Plant Family', 'Native Status', 'unique_id']]

eco_json = eco_subset.to_json(orient='records', force_ascii=False)
repr(eco_json)

file = open("eco_county_397.json", "w")
file.write(eco_json)
file.close


In [ ]:

# j_test = pd.read_json('/home/muskrat/Documents/eco_data_copy/US_county_plants/397/eco_county_397.json')

 
# USA ECO COUNTIES

# read in all json files and merge into one dataframe
json_dir = '/home/muskrat/Documents/eco_data_copy/US_county_plants/eco_json'

json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
counties_init = pd.concat(dfs)
counties_init.reset_index(drop=True, inplace=True)
counties_init.drop('site', 1, inplace=True)

 

counties_init['Plant Type'].replace(
    to_replace='Tree, Shrub', value='Tree', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Herb (annual)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Herb (perennial)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Shrub, Herb (perennial)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Graminoid (perennial)', value='Graminoid', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Graminoid (annual)', value='Graminoid', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Herb (annual or perennial)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Herb (biennial or perennial)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Herb (biennial)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Herb (annual or biennial)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Shrub, Herb (annual or perennial)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Graminoid (annual or perennial)', value='Graminoid', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Shrub, Herb (biennial or perennial)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Shrub, Herb (annual)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Shrub (annual or perennial)', value='Shrub', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Herb', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Vine, Herb (annual or perennial)', value='Vine', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Vine, Herb (perennial)', value='Vine', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Shrub, Graminoid (perennial)', value='Graminoid', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Shrub, Herb (annual or perennial)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Vine, Herb (annual)', value='Vine', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Vine, Shrub', value='Vine', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Vine, Shrub, Herb (perennial)', value='Vine', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Shrub, Herb (biennial or perennial)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Tree, Vine, Shrub', value='Shrub', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Tree, Shrub, Herb (perennial)', value='Wildflower', inplace=True)
counties_init['Plant Type'].replace(
    to_replace='Vine, Shrub, Graminoid (perennial)', value='Graminoid', inplace=True)

counties_init.loc[77973, 'Plant Type'] = 'Tree'
counties_init.loc[93777, 'Plant Type'] = 'Tree'

 
counties_unique = counties_init.drop_duplicates(['Scientific Name'])
counties_unique.drop('unique_id', 1, inplace=True)
counties_group = counties_init.groupby('Scientific Name')[
    'unique_id'].apply(list).reset_index()

counties_final = pd.merge(counties_unique, counties_group,
                          on='Scientific Name', how='left')
common_miss = counties_final.loc[pd.isna(counties_final["Common Name"]), :]

 

final_json = counties_final.to_json(orient='records', force_ascii=False)
repr(final_json)

final_miss = common_miss.to_json(orient='records', force_ascii=False)
repr(final_miss)

file = open("eco_county_prep.json", "w")
file.write(final_json)
file.close


In [ ]:
# USA ECO COUNTIES

eco_county = pd.read_json('/home/muskrat/Desktop/eco_county_prep.json')

# convert to dataframe and group county and state into dict.
county = ['county', 'state']
eco_county['eco_counties'] = eco_county[county].to_dict(orient='records')

# group by unique id
eco_group = eco_county.groupby(
    'unique_id')['eco_counties'].apply(list).reset_index()
# eco_group = eco_county.groupby('unique_id')['eco_counties']

# convert to list
eco_counties = eco_group['eco_counties'].to_list()

eco_counties_test = eco_counties

# remove duplicates from list of counties and states
for j in range(len(eco_counties_test)):

    eco_counties_test[j] = [i for n, i in enumerate(
        eco_counties_test[j]) if i not in eco_counties_test[j][n + 1:]]


eco_counties_df = pd.Series(eco_counties_test, name='counties_final')

group_final = eco_group.merge(eco_counties_df, left_index=True, right_index=True).reindex(
    columns=['unique_id', 'counties_final'])

group_final_json = group_final.to_json(orient='records', force_ascii=False)
repr(group_final_json)

file = open("eco_county_within.json", "w")
file.write(group_final_json)
file.close



